In [6]:
##### this model is the model_2 in our report executed on deceptive corpus"###############

In [1]:
import os
import io
import re
import matplotlib.pyplot as plt
import gensim
from six.moves import cPickle as pickle
import numpy as np
import scipy.stats as stats
import pandas as pd
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
truthful_pos = 'op_spam_v1.4/positive_polarity/truthful_from_TripAdvisor/'
truthful_neg = 'op_spam_v1.4/negative_polarity/truthful_from_Web/'

deceptive_pos = 'op_spam_v1.4/positive_polarity/deceptive_from_MTurk/'
deceptive_neg = 'op_spam_v1.4/negative_polarity/deceptive_from_MTurk/'

truthful_reviews_link = []
for fold in os.listdir(truthful_pos):
    foldLink = os.path.join(truthful_pos, fold)
    if os.path.isdir(foldLink):
        for f in os.listdir(foldLink):
            fileLink = os.path.join(foldLink, f)
            truthful_reviews_link.append(fileLink)

for fold in os.listdir(truthful_neg):
    foldLink = os.path.join(truthful_neg, fold)
    if os.path.isdir(foldLink):
        for f in os.listdir(foldLink):
            fileLink = os.path.join(foldLink, f)
            truthful_reviews_link.append(fileLink)

deceptive_reviews_link = []

for fold in os.listdir(deceptive_pos):
    foldLink = os.path.join(deceptive_pos, fold)
    if os.path.isdir(foldLink):
        for f in os.listdir(foldLink):
            fileLink = os.path.join(foldLink, f)
            deceptive_reviews_link.append(fileLink)

for fold in os.listdir(deceptive_neg):
    foldLink = os.path.join(deceptive_neg, fold)
    if os.path.isdir(foldLink):
        for f in os.listdir(foldLink):
            fileLink = os.path.join(foldLink, f)
            deceptive_reviews_link.append(fileLink)
        
print('Number of truthfuls reviews ', len(truthful_reviews_link))
print('Number of deceptives reviews ', len(deceptive_reviews_link))

Number of truthfuls reviews  800
Number of deceptives reviews  800


In [3]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

def handleFile(filePath):
    with open(filePath, "r") as f:
        lines=f.readlines()
        file_voc = []
        file_numWords = 0
        for line in lines:
            cleanedLine = clean_str(line)
            cleanedLine = cleanedLine.strip()
            cleanedLine = cleanedLine.lower()
            words = cleanedLine.split(' ')
            file_numWords = file_numWords + len(words)
            file_voc.extend(words)
    return file_voc, file_numWords


allFilesLinks = truthful_reviews_link + deceptive_reviews_link
vocabulary = []
numWords = []
for fileLink in allFilesLinks:
    file_voc, file_numWords = handleFile(fileLink)
    vocabulary.extend(file_voc)
    numWords.append(file_numWords)

vocabulary = set(vocabulary)
vocabulary = list(vocabulary)

print('The total number of files is ', len(numWords))
print('The total number of words in the files is ', sum(numWords))
print('Vocabulary size is ', len(vocabulary))
print('The average number of words in the files is', sum(numWords)/len(numWords))

The total number of files is  1600
The total number of words in the files is  253157
Vocabulary size is  9687
The average number of words in the files is 158.223125


In [4]:
MAX_SEQ_LENGTH = 200

def convertFileToArray(filePath):
    s = ""
    with open(filePath, "r") as f:
        lines=f.readlines()
        for line in lines:
            cleanedLine = clean_str(line)
            cleanedLine = cleanedLine.strip()
            cleanedLine = cleanedLine.lower()
            s += cleanedLine
    return s

totalFiles = len(truthful_reviews_link) + len(deceptive_reviews_link)
idsMatrix = np.ndarray(shape=(totalFiles, MAX_SEQ_LENGTH), dtype='int32')
#dataMatrix = np.ndarray(shape=(totalFiles,1),dtype='object')
dataMatrix = []
#labels = np.ndarray(shape=(totalFiles, 2), dtype='int32')
labelsMatrix = []
counter = 0

for filePath in truthful_reviews_link:
    dataMatrix.append(convertFileToArray(filePath))
    labelsMatrix.append(1)
  

for filePath in deceptive_reviews_link:
    dataMatrix.append(convertFileToArray(filePath))
    labelsMatrix.append(0)

dict_reviewLabels = {'review': dataMatrix,'labels': labelsMatrix}
df_reviewLabels = pd.DataFrame(dict_reviewLabels)
df_reviewLabels.head(2)



macronum=sorted(set(df_reviewLabels['labels']))
macro_to_id = dict((note, number) for number, note in enumerate(macronum))


def fun(i):
    return macro_to_id[i]

pd.set_option('mode.chained_assignment',None)
df_reviewLabels.iloc[:,0]=df_reviewLabels.iloc[:,0].apply(fun)
df_reviewLabels.head(2)



print(df_reviewLabels.shape)
# a list contains each review as a list 
balanced_texts = []
balanced_labels = []

for i in range(len(df_reviewLabels)):
    balanced_texts.append(df_reviewLabels.iloc[i,1])
    balanced_labels.append(df_reviewLabels.iloc[i,0])
 


(1600, 2)


In [9]:
from nltk import tokenize
import re
from keras.preprocessing.text import text_to_word_sequence
import nltk
nltk.download('punkt')
#parameters for hierarchical attention network
MAX_SENT_LENGTH = 100
MAX_SENTS = 15
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2
"""
train_han = train.dropna()
train_han =train_han.reset_index(drop=True)
print('Shape of dataset ',train_han.shape)
print(train_han.columns)
print('No. of unique classes',len(set(train_han['label'])))


macronum=sorted(set(train_han['label']))
macro_to_id = dict((note, number) for number, note in enumerate(macronum))


def fun(i):
    return macro_to_id[i]

pd.set_option('mode.chained_assignment',None)
train_han.iloc[:,0]=train_han.iloc[:,0].apply(fun)
# a list contains each review as a list 
#balanced_texts = []
#balanced_labels = []

#for i in range(len(train)):
   # balanced_texts.append(balanced_train.iloc[i,1])
   # balanced_labels.append(balanced_train.iloc[i,0])
"""
texts = []
labels = []
reviews = []

def clean_str(string):
    string = re.sub(r"\\", "", string)
    string = re.sub(r"\'", "", string)
    string = re.sub(r"\"", "", string)
    return string.strip().lower()

for i in range(df_reviewLabels.review.shape[0]):
    #text = BeautifulSoup(df.message[i])
    text=clean_str(df_reviewLabels.iloc[i,1].lower())
    texts.append(text)
    sentences = tokenize.sent_tokenize(text)
    reviews.append(sentences)


[nltk_data] Downloading package punkt to /home/nhumair/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)

data = np.zeros((len(texts),MAX_SENTS, MAX_SENT_LENGTH), dtype= 'int32')


In [11]:
for i, sentences in enumerate(reviews):
    for j, sent in enumerate(sentences):
        if j < MAX_SENTS:
            wordTokens = text_to_word_sequence(sent)
            k = 0
            for _, word in enumerate(wordTokens):
                try:
                    if k < MAX_SENT_LENGTH and tokenizer.word_index[word] < MAX_NB_WORDS:
                        data[i,j,k] = tokenizer.word_index[word]
                        k = k+1
                except KeyError:
                    pass

In [12]:
word_index = tokenizer.word_index
print('No of %s unique tokens.' % len(word_index))

No of 9597 unique tokens.


In [15]:
from keras.utils.np_utils import to_categorical
labels = []
for i in df_reviewLabels['labels']:
    labels.append(i)
labels_n = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of data tensor:', labels_n.shape)

Shape of data tensor: (1600, 15, 100)
Shape of data tensor: (1600, 2)


In [16]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels_n = labels_n[indices]
nb_validation_samples = int(0.2 * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels_n[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels_n[-nb_validation_samples:]


In [18]:
import os
embeddings_index = {}
cur_dir = os.getcwd()
f = open(os.path.join(cur_dir, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [20]:
from keras.layers import Embedding
#prepare embedding matrix
EMBEDDING_DIM = 100
#num_words = min(20000, len(word_index)) + 1
#embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
'''
for word, i in word_index.items():
    if i > 20000:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
'''
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,weights=[embedding_matrix],
                            input_length=MAX_SENT_LENGTH,trainable=False)


In [22]:
from keras.layers import Input, Bidirectional, TimeDistributed, Flatten, Dense, LSTM
from keras.models import Model
sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype = 'int32')
embedded_sequences = embedding_layer(sentence_input)
l_lstm = Bidirectional(LSTM(100))(embedded_sequences)
sentEncoder = Model(sentence_input, l_lstm)

review_input = Input(shape = (MAX_SENTS,MAX_SENT_LENGTH), dtype='int32')
review_encoder = TimeDistributed(sentEncoder)(review_input)
l_lstm_sent = Bidirectional(LSTM(100))(review_encoder)
preds = Dense(len(macronum),activation='softmax')(l_lstm_sent)
model = Model(review_input, preds)

model.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 15, 100)           0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 15, 200)           1120600   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 200)               240800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 402       
Total params: 1,361,802
Trainable params: 402,002
Non-trainable params: 959,800
_________________________________________________________________


In [25]:
history = model.fit(x_train,y_train, validation_data = (x_val, y_val),epochs=45,batch_size=10)

Train on 1280 samples, validate on 320 samples
Epoch 1/45
1280/1280 [==============================] - 56s 44ms/step - loss: 0.0776 - acc: 0.9742 - val_loss: 1.0284 - val_acc: 0.7719
Epoch 2/45
1280/1280 [==============================] - 56s 43ms/step - loss: 0.0579 - acc: 0.9805 - val_loss: 1.1791 - val_acc: 0.7094
Epoch 3/45
1280/1280 [==============================] - 56s 44ms/step - loss: 0.0662 - acc: 0.9742 - val_loss: 1.0319 - val_acc: 0.7844
Epoch 4/45
1280/1280 [==============================] - 56s 43ms/step - loss: 0.0582 - acc: 0.9805 - val_loss: 1.0618 - val_acc: 0.7656
Epoch 5/45
1280/1280 [==============================] - 56s 44ms/step - loss: 0.0490 - acc: 0.9836 - val_loss: 1.0986 - val_acc: 0.7781
Epoch 6/45
1280/1280 [==============================] - 56s 44ms/step - loss: 0.0542 - acc: 0.9828 - val_loss: 0.9701 - val_acc: 0.7500
Epoch 7/45
1280/1280 [==============================] - 56s 44ms/step - loss: 0.0403 - acc: 0.9852 - val_loss: 1.0120 - val_acc: 0.8031
E

KeyboardInterrupt: 

In [26]:
loss, accuracy = model.evaluate(x_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(x_val, y_val, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.9906
Testing Accuracy:  0.7469
